In [2]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Get GPU device name
    gpu_device_name = gpus[0].name
    print("GPU device available:", gpu_device_name)
else:
    print("No GPU device available.")


TensorFlow version: 2.16.2
GPU device available: /physical_device:GPU:0


2024-07-10 18:22:07.970211: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 18:22:07.974901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 18:22:07.975067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
!nvidia-smi

Wed Jul 10 18:00:32 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650        Off |   00000000:01:00.0  On |                  N/A |
| 23%   33C    P8              4W /   75W |     339MiB /   4096MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import tensorflow as tf
import os

# Define image data generators for train, validation, and test sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # if splitting into train and validation sets
)

# Define the directory containing training images
train_data_directory = '/desktop/ASL_Dataset/Train'
test_data_directory = '/desktop/ASL_Dataset/Test'

# Define the image size
img_height, img_width = 128, 128  # or whatever size you prefer

batch_size = 32

num_classes = len(os.listdir(train_data_directory))

#flow_from_directory method reads the data automatically inside the sub folders.

train_generator = train_datagen.flow_from_directory(
    directory=train_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # specify if using validation_split
)

validation_generator = train_datagen.flow_from_directory(
    directory=train_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # specify if using validation_split
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory=test_data_directory,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


2024-07-10 18:22:00.213230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-10 18:22:00.229297: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-10 18:22:00.229330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-10 18:22:00.239198: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 132542 images belonging to 28 classes.
Found 33128 images belonging to 28 classes.
Found 112 images belonging to 28 classes.


In [5]:
!pip install scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 15.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
import os

# # Define data directories
# train_data_dir = 'path_to_train_data'
# validation_data_dir = 'path_to_validation_data'

# Check the number of classes
num_classes_train = len(os.listdir(train_data_directory))
num_classes_val = len(os.listdir(test_data_directory))

print("Number of classes in training data:", num_classes_train)
print("Number of classes in validation data:", num_classes_val)


Number of classes in training data: 28
Number of classes in validation data: 28


In [10]:

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



num_classes = 28 # Set the correct number of classes in your dataset

# Define the number of epochs
num_epochs = 10  # You can adjust this number based on your training needs

# Fit the model to the training data
history = model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator
)



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1720636446.659955     190 service.cc:145] XLA service 0x7b82580042d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720636446.659980     190 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-07-10 18:34:06.736765: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-10 18:34:06.999590: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


   3/4142 ━━━━━━━━━━━━━━━━━━━━ 5:09 75ms/step - accuracy: 0.0451 - loss: 3.3543 

I0000 00:00:1720636451.080184     190 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4142/4142 ━━━━━━━━━━━━━━━━━━━━ 567s 135ms/step - accuracy: 0.6630 - loss: 1.0672 - val_accuracy: 0.9467 - val_loss: 0.1569
Epoch 2/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 496s 120ms/step - accuracy: 0.9737 - loss: 0.0822 - val_accuracy: 0.9472 - val_loss: 0.1700
Epoch 3/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 495s 119ms/step - accuracy: 0.9810 - loss: 0.0579 - val_accuracy: 0.9804 - val_loss: 0.0643
Epoch 4/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 496s 120ms/step - accuracy: 0.9871 - loss: 0.0424 - val_accuracy: 0.9859 - val_loss: 0.0469
Epoch 5/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 500s 121ms/step - accuracy: 0.9890 - loss: 0.0384 - val_accuracy: 0.9781 - val_loss: 0.0726
Epoch 6/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 504s 121ms/step - accuracy: 0.9904 - loss: 0.0340 - val_accuracy: 0.9877 - val_loss: 0.0395
Epoch 7/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 501s 121ms/step - accuracy: 0.9912 - loss: 0.0322 - val_accuracy: 0.9843 - val_loss: 0.0647
Epoch 8/10
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 500s 121ms/step - accuracy: 0.9

In [12]:
# Assuming `model` is your trained model
model.save('/desktop/Miniproject/model.h5')
